In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

import keras
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, MaxPooling2D, Conv2D, AveragePooling2D, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [ ]:
from keras.datasets import cifar10
from keras.utils import np_utils

In [ ]:
(xx_train, yy_train), (x_test, y_test) = cifar10.load_data()
x_train = xx_train[:40000]
y_train = yy_train[:40000]
x_valid = xx_train[40000:50000]
y_valid = yy_train[40000:50000]

x_train = x_train/255.0
x_valid = x_valid/255.0
x_test = x_test/255.0

y_train = np_utils.to_categorical(y_train)
y_valid = np_utils.to_categorical(y_valid)
y_test = np_utils.to_categorical(y_test)

In [ ]:
x_train.shape

In [ ]:
x_train.shape[1:]

In [2]:
batch_size = 32
dim = 150
datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_gen = datagen.flow_from_directory('dogscats/train',
                                       target_size=(dim,dim),
                                       batch_size=batch_size,
                                       class_mode='binary')

valid_gen = test_datagen.flow_from_directory('dogscats/valid',
                                       target_size=(dim,dim),
                                       batch_size=batch_size,
                                       class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [3]:
l = Input((dim, dim, 3))
pool = MaxPooling2D(pool_size=(3,3), strides=2)

In [4]:
def resnet_block(conv_val, x, repeat_num):
#     input_layer = pool(input_layer)
#     conv = Conv2D(conv_val, (3,3), padding='same')
#     input_layer_tmp = conv(input_layer)
#     for i in xrange(repeat_num-1):
#         input_layer_tmp = conv(input_layer_tmp)
#     input_layer = Add()([input_layer,input_layer_tmp])
#     return Activation('relu')(input_layer)
    x = pool(x)
    conv = Conv2D(conv_val, (3,3), strides=1, padding='same', kernel_initializer='he_normal')
    for i in xrange(repeat_num):
        xtmp = BatchNormalization()(x)
        xtmp = Activation('relu')(xtmp)
        xtmp = conv(xtmp)
    x = Concatenate()([x,xtmp])
    return x

In [5]:
from keras.layers import BatchNormalization, Concatenate, Add, Flatten, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [6]:
x = Conv2D(64, (7,7), strides=2, padding='same', kernel_initializer='he_normal')(l)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = resnet_block(64, x, 2)
x = resnet_block(128, x, 2)
x = resnet_block(256, x, 2)
x = resnet_block(512, x, 2)
# x = pool(x)
# conv = Conv2D(128, (3,3), strides=1, padding='same', kernel_initializer='he_normal')
# xtmp = BatchNormalization()(x)
# xtmp = Activation('relu')(xtmp)
# xtmp = conv(xtmp)
# xtmp = BatchNormalization()(xtmp)
# xtmp = Activation('relu')(xtmp)
# xtmp = conv(xtmp)
# x = Concatenate()([x,xtmp])

# x = pool(x)
# conv = Conv2D(256, (3,3), strides=1, padding='same', kernel_initializer='he_normal')
# xtmp = BatchNormalization()(x)
# xtmp = Activation('relu')(xtmp)
# xtmp = conv(xtmp)
# xtmp = BatchNormalization()(xtmp)
# xtmp = Activation('relu')(xtmp)
# xtmp = conv(xtmp)
# x = Concatenate()([x,xtmp])

# x = pool(x)
# conv = Conv2D(512, (3,3), strides=1, padding='same', kernel_initializer='he_normal')
# xtmp = BatchNormalization()(x)
# xtmp = Activation('relu')(xtmp)
# xtmp = conv(xtmp)
# xtmp = BatchNormalization()(xtmp)
# xtmp = Activation('relu')(xtmp)
# xtmp = conv(xtmp)
# x = Concatenate()([x,xtmp])

# x = Flatten()(x)

# x = AveragePooling2D(pool_size=(2,2))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D()(x)
# x = Dense(1, activation='sigmoid')(x)
# x = Dense(10, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)
x = Dense(1, activation='sigmoid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0001))(x)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
model = load_model('weights.best.hdf5')

# model.compile(optimizer='adam', 
#               loss='binary_crossentropy', 
#               metrics=['accuracy'])

early=EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto')
filepath="weights.best.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
model = Model(l,x)
# model.compile(optimizer='adam', 
#               loss='sparse_categorical_crossentropy', 
#               metrics=['accuracy'])

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
early=EarlyStopping(monitor='val_acc', patience=15, verbose=1, mode='auto')
filepath="weights.best.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.summary()

model.fit(x_train,y_train,
          batch_size=2000, epochs=50,
          shuffle=True,
         validation_data=(x_valid, y_valid), callbacks=[early])
model.save_weights('model.h5')

In [8]:
model.fit_generator(
        train_gen,
        steps_per_epoch=2000 // batch_size,
        epochs=150,
        validation_data=valid_gen,
        validation_steps=800 // batch_size,
        callbacks=[early,checkpoint],
        shuffle=True,
        verbose=1)

Epoch 1/150
62/62 [==============================] - 98s 2s/step - loss: 0.2002 - acc: 0.9215 - val_loss: 0.2398 - val_acc: 0.8938
Epoch 2/150
62/62 [==============================] - 92s 1s/step - loss: 0.1967 - acc: 0.9189 - val_loss: 0.2192 - val_acc: 0.9075
Epoch 3/150
62/62 [==============================] - 90s 1s/step - loss: 0.2109 - acc: 0.9194 - val_loss: 0.4082 - val_acc: 0.8275
Epoch 4/150
62/62 [==============================] - 89s 1s/step - loss: 0.2303 - acc: 0.8997 - val_loss: 0.3553 - val_acc: 0.8550
Epoch 5/150
62/62 [==============================] - 90s 1s/step - loss: 0.2173 - acc: 0.9173 - val_loss: 0.2987 - val_acc: 0.8738
Epoch 6/150
62/62 [==============================] - 89s 1s/step - loss: 0.2271 - acc: 0.9012 - val_loss: 0.2131 - val_acc: 0.9025
Epoch 7/150
62/62 [==============================] - 91s 1s/step - loss: 0.2220 - acc: 0.8967 - val_loss: 0.2247 - val_acc: 0.9087
Epoch 8/150
62/62 [==============================] - 90s 1s/step - loss: 0.2088 - a